In [1]:
import pandas as pd
import numpy as np

# 1. Criar base de dados fictícia
np.random.seed(42)
data = {
    "ID_Transação": range(1, 101),
    "Valor_Transação": np.random.uniform(50, 500, 100).round(2),
    "Taxa_Cobrada": np.random.uniform(0, 5, 100).round(2),
    "Isenção_Aplicada": np.random.choice(["Sim", "Não"], 100, p=[0.2, 0.8])
}
df = pd.DataFrame(data)

# 2. Simular erros e anomalias
# 2.1. Corrigir isenções (taxa = 0) e adicionar erros de cobrança
df.loc[df["Isenção_Aplicada"] == "Sim", "Taxa_Cobrada"] = 0
erros_isenção = np.random.choice(df[df["Isenção_Aplicada"] == "Sim"].index, 5, replace=False)
df.loc[erros_isenção, "Taxa_Cobrada"] = np.random.uniform(1, 10, 5).round(2)

# 2.2. GERAR ANOMALIAS (outliers) em transações NÃO isentas
anomalias = np.random.choice(df[df["Isenção_Aplicada"] == "Não"].index, 5, replace=False)
df.loc[anomalias, "Taxa_Cobrada"] = np.random.choice([25, 26, 27, 28, 38], 5)  # Valores extremos

# 3. Salvar em CSV (arquivo será criado no mesmo diretório do script)
df.to_csv("transacoes_paysmart.csv", index=False, encoding='utf-8-sig')

print("✅ Arquivo 'transacoes_paysmart.csv' gerado com sucesso!")

✅ Arquivo 'transacoes_paysmart.csv' gerado com sucesso!


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
#import schedule
import time

def monitorar_cobrancas():
    # 1. Carregar base de dados (simulada ou real)
    df = pd.read_csv("transacoes_paysmart.csv")  # Ou conexão com banco de dados

    # 2. Calcular métricas e identificar problemas
    df["Z-Score"] = (df["Taxa_Cobrada"] - df["Taxa_Cobrada"].mean()) / df["Taxa_Cobrada"].std()
    df["Anomalia"] = np.abs(df["Z-Score"]) > 3
    df["Erro_Isencao"] = (df["Isenção_Aplicada"] == "Sim") & (df["Taxa_Cobrada"] > 0)
    df["Problema"] = df["Anomalia"] | df["Erro_Isencao"]

    # 3. Filtrar transações problemáticas
    problemas = df[df["Problema"]]

    # 4. Gerar relatório e alertas
    if not problemas.empty:
        data_hora = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        problemas.to_csv(f"alertas_cobrancas_{data_hora}.csv", index=False)
        print(f"⚠️ {len(problemas)} alertas gerados em {data_hora}. Verifique o arquivo CSV.")
    else:
        print(f"✅ Nenhum problema detectado em {datetime.now()}.")


In [3]:
monitorar_cobrancas()

⚠️ 10 alertas gerados em 2025-04-20_16-33-50. Verifique o arquivo CSV.
